In [1]:
import pandas as pd
import numpy as np
import json

from funciones_varias import desanidarColumna, desanidarVariasColumnas, creaDfFuncionesFastAPI

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets_crudos\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets_crudos\credits.csv', dtype='unicode')

In [2]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])
except:
    print("estas transformaciones ya se ejecutaron")


In [3]:
#desanido belongs to collection
franquicia = (desanidarColumna(dfMovies['belongs_to_collection']))

#inserto nueva columna y elimino la vieja
dfMovies= dfMovies.drop(columns=["belongs_to_collection"])
dfMovies.insert(1, "franquicia", franquicia["name"])

In [4]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion      
dfGeneros=dfGeneros.iloc[:45378] #esto es cualquier cosa pero bueno, funciona xd
                                 #uso iloc para obtener solo hasta el row 45378 porque sino tengo errores
                                 
#inserto nueva columna y elimino la vieja
dfMovies.insert(1, "movie_genre", dfGeneros["name"])
dfMovies= dfMovies.drop(columns=["genres"])

In [5]:
#desanido tabla production_companies
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies= dfCompanies.iloc[:45379] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(7, "production_company", dfCompanies["name"])
dfMovies= dfMovies.drop(columns=["production_companies"])

In [6]:
#desanido production_countries
dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries = dfProdCountries.iloc[:45379] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(8, "production_country", dfProdCountries["name"])
dfMovies= dfMovies.drop(columns=["production_countries"])

In [7]:
#desanido spoken_languages
dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan = dfSpokenLan.iloc[:45379] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(12, "language", dfSpokenLan["name"]) #puedo elegir entre poner la abreviacion o la palabra entera
dfMovies= dfMovies.drop(columns=["spoken_languages"])

In [8]:
#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]
#cambio valores de fechas invalidas a fechas default
release_year[19730] = "1900-01-01"
release_year[29503] = "1900-01-01"
release_year[35587] = "1900-01-01"

#obtengo solo año de cada string de fecha y lo meto en release_year
release_year = pd.to_datetime(release_year).dt.year

#inserto nueva columan release_year
dfMovies.insert(10, "release_year", release_year)

#se supone que transforme la columna en date type pero es un int REVISAR

In [9]:
#comienzo a armar columna return

budget=dfMovies["budget"] #debo reemplazar esos valores porque sino me da error
budget[19730] = 0
budget[29503] = 0
budget[35587] = 0

#paso a float
dfMovies["revenue"]=pd.DataFrame(dfMovies["revenue"]).astype(float)
dfMovies["budget"]=pd.DataFrame(dfMovies["budget"]).astype(float)

#creo la columna return y relleno con 0 cuando el resultado es None
dfMovies["return"]=round(dfMovies["revenue"]/dfMovies["budget"], 2)
dfMovies['return'] = dfMovies['return'].fillna(0)

#reemplazo valores infinitos
dfMovies.replace([np.inf, -np.inf], 0, inplace=True)

#para revisar si hay valores infinitos
counter=0
for i in dfMovies["return"]:
    if i == float("inf"):
        print(i)
        counter+=1
print(counter)


0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\395374050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[19730] = 0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\395374050.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[29503] = 0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\395374050.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[35587] = 0


In [10]:
#desanido la columna crew y la dejo en un df separado (tarda como 13 minutos en armarse)

#dfCrew = pd.DataFrame()
#for i in range(len(dfCredits)):
#    stringsCrew = dfCredits.crew[i]
#    dictionaryCrew = eval(stringsCrew) #esto es para pasarlo de string a lista y poder armar el df
#    dictionaryCrew = pd.DataFrame(dictionaryCrew)
#    dictionaryCrew["id"] = dfCredits.id[i]  #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
#    dfCrew = pd.concat([dfCrew, dictionaryCrew])
    #dfCrew = dfCrew.reset_index(drop=True) #si descomento el reindex tarda mas del doble

In [11]:
#desanido la columna cast y la dejo en un df separado (tarda como 17 minutos en armarse)
#dfCast = pd.DataFrame()
#for i in range(len(dfCredits)):
#    stringsCast = dfCredits.cast[i]
#    dictionaryCast = eval(stringsCast) #esto es para pasarlo de string a lista y poder armar el df
#    dictionaryCast = pd.DataFrame(dictionaryCast)
#    dictionaryCast["id"] = dfCredits.id[i] #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
#    dfCast = pd.concat([dfCast, dictionaryCast])
    #dfCast = dfCast.reset_index(drop=True) #si descomento el reindex tarda mas del doble

In [12]:
#cambio de posicion la columna "title" y le cambio el nombre a "movie_title"
titles=pd.DataFrame(dfMovies["title"])
dfMovies.insert(0,"movie_title" ,titles)
dfMovies=dfMovies.drop(columns=["title"])

#cambio nans en todas las columnas por "Unkown Value"

#columns = [a for a in dfMovies.columns]
#for i in range(len(columns)):
#    dfMovies[columns[i]]=dfMovies[columns[i]].fillna("Unkown Value")

#valores raros en id
id=dfMovies["id"]

id[19730] = 1111111
id[29503] = 2222222
id[35587] = 3333333


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\3565241362.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id[19730] = 1111111
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\3565241362.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id[29503] = 2222222
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7208\3565241362.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id[35587] = 3333333


In [13]:
#para exportar csv e ir probando cargarlo a la API
#dfMovies.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMovies.csv")

In [14]:
#dfMovies["release_date"]
                                                      
#dfMovies["release_date"] = pd.to_datetime(dfMovies["release_date"], format="%Y-%m-%d")
#month = dfMovies["release_date"].dt.month_name()
#dfMovies[""]


#"710": "710", REVISAR ESTE ID, INDEX NO SE. A PARTIR DE ACA YA NO COINCIDEN EL ID O ALGO ASI.

In [15]:
#PRIMERA FUNCION DATAFRAME dfPeliculasIdiomas

#dataset funcion peliculas idioma
#dfMoviesDuration=creaDfFuncionesFastAPI(["movie_title", "language", "original_language"], dfMovies)

#guardo dataset para la primera funcion
#dfPeliculasIdiomas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPeliculasIdiomas.csv")

In [16]:
#SEGUNDA FUNCION DATAFRAME dfMoviesDuration

#nuevo df con la info que necesito para la segunda funcion
#dfMoviesDuration=creaDfFuncionesFastAPI(["movie_title", "release_year", "runtime"], dfMovies)

#guardo dataset para la segunda funcion
#dfMoviesDuration.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMoviesDuration.csv")

In [17]:
#TERCERA FUNCION DATAFRAME dfFranquicia

#POR EL MOMENTO, GANANCIA TOTAL VA A SER GIUAL A REVENUE
#dfMovies["ganancia_total"]=round(dfMovies["revenue"]-dfMovies["budget"], 2)

#pd.options.display.float_format = '{:.2f}'.format #porque sino me tira numeros con notacion cientifica

#solo obtengo 5381 registros ya que hay muchos campos nulos tanto en budget como en revenue
#dfMovies.loc[(dfMovies["revenue"] > 0) & (dfMovies["budget"] > 0) , ["movie_title","franquicia","budget","revenue", "ganancia_total"]]

#la funcion toma como parametro FRANQUICIA
#necesito CANTIDAD de peliculas de esa franquicia
#GANACIA total (revenue)
#PROMEDIO dela ganancia total

#filtro revenue mayor a 0 y me quedo con columnas que necesito
#dfFiltrado = dfMovies.loc[(dfMovies["revenue"] > 0), ["franquicia","movie_title","revenue"]]

#hago group by por franquicia y agrego cada una de las maneras para agrupar las demas columnas
#dfFranquicia = dfFiltrado.groupby("franquicia").agg(
#    movie_count=("movie_title", "count"),
#    ganancia_total=("revenue", "sum"),
#    ganancia_promedio=("revenue", "mean")).reset_index()

#guardo dataset para la tercera funcion
#dfFranquicia.to_csv(r"C:\Users\\Administrator\\Desktop\\dfFranquicia.csv")




In [18]:
#CUARTA FUNCION DATAFRAME dfPaises

#def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!),
# retornando la cantidad de peliculas producidas en el mismo.
#    Ejemplo de retorno: Se produjeron X películas en el país X

#creo df para la cuarta funcion
#dfPaises = creaDfFuncionesFastAPI(["production_country", "movie_title"])

#agrupo por pais y cuento las peliculas
#dfPaises = dfPaises.groupby("production_country").agg(
#    movie_count=("movie_title", "count")).reset_index()

#exporto df para cuarta funcion
#dfPaises.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPaises.csv")


In [19]:
#QUINTA FUNCION DATAFRAME dfProductora

#def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la
# cantidad de peliculas que realizo.
#Ejemplo de retorno: La productora X ha tenido un revenue de x

#dfProductora = creaDfFuncionesFastAPI(["production_company", "movie_title","revenue"], dfMovies)

#dfProductora = dfMovies.loc[(dfMovies["revenue"] > 0), ["production_company","movie_title","revenue"]]

#dfProductora = dfProductora.groupby("production_company").agg(
#    movie_count =("movie_title", "count"),
#    ganancia_total = ("revenue", "sum")
#).reset_index()

#exporto df para quinta funcion
#dfProductora.to_csv(r"C:\Users\\Administrator\\Desktop\\dfProductora.csv")

In [20]:
#armo dfs que voy a usar para la sexta funcion

#dfMovieReturn = creaDfFuncionesFastAPI(["id", "return"], dfMovies)

#dfMovieInfo = creaDfFuncionesFastAPI(["id", "movie_title","release_date", "budget","revenue","return"], dfMovies)

#dfDirectores = dfCrew.loc[(dfCrew["job"] == "Director"), ["id","job","name"]]

In [21]:
#SEXTA FUNCION DATAFRAME 1 dfDirectoresExito

#filtro para no traer los returns en 0
#dfMovieReturn = dfMovieReturn.loc[dfMovieReturn["return"]>0]

#para revisar cuantos directores hay por pelicula
#dfDirectoresPorPelicula = dfDirectores.groupby("id").agg(
#    job_count=("job", "count"),
#    name_count=("name", "count")
#).reset_index()
#dfDirectoresPorPelicula[dfDirectoresPorPelicula["job_count"]>1]

#hago join para poder calcular el return de cada director
#dfDirectoresExito = dfDirectores.merge(dfMovieReturn, on="id",how="left")

#dropeo columnas que no me sirven
#dfDirectoresExito=dfDirectoresExito.drop(columns=["job", "id"])

#hago group by director name
#dfDirectoresExito = dfDirectoresExito.groupby("name").agg(
#    director_return = ("return", "mean")
#)

#resultado sin tener en cuenta cuando no hay return diponible para el director
#dfDirectoresExito = dfDirectoresExito.loc[dfDirectoresExito["director_return"]>0]

#exporto uno de los dfs que voy a usar para la sexta funcion
#dfDirectoresExito.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresExito.csv")

In [22]:
#SEXTA FUNCION DATAFRAME 2 dfDirectoresInfoPeliculas

#filtro para no traer valores en 0
#dfMovieInfo = dfMovieInfo.loc[dfMovieInfo["return"]>0]
#hago join para poder calcular el return de cada director
#dfDirectoresInfoPeliculas = dfMovieInfo.merge(dfDirectores, on="id",how="left")

#hago reindex
#dfDirectoresInfoPeliculas = dfDirectoresInfoPeliculas.reindex(columns=["id","name", "job","movie_title", "release_date",
#                                                                "budget", "revenue", "return"])
#dropeo columnas que no me sirven
#dfDirectoresInfoPeliculas=dfDirectoresInfoPeliculas.drop(columns=["job", "id"])                                                                 

#exporto el otro de los dfs que voy a usar para la sexta funcion
#dfDirectoresInfoPeliculas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresInfoPeliculas.csv")        

In [24]:
dfMovies

,movie_title,budget,movie_genre,franquicia,id,original_language,overview,popularity,production_company,production_country,release_date,release_year,revenue,runtime,language,status,tagline,vote_average,vote_count,return
0,Toy Story,30000000.0,Animation,Toy Story Collection,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,1995,373554033.0,81.0,English,Released,NaN,7.7,5415,12.45
1,Jumanji,65000000.0,Adventure,NaN,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,1995,262797249.0,104.0,English,Released,Roll the dice and unleash the excitement!,6.9,2413,4.04
2,Grumpier Old Men,0.0,Romance,Grumpy Old Men Collection,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,United States of America,1995-12-22,1995,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,6.5,92,0.00
3,Waiting to Exhale,16000000.0,Comedy,NaN,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,1995,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,6.1,34,5.09
4,Father of the Bride Part II,0.0,Comedy,Father of the Bride Collection,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,United States of America,1995-02-10,1995,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,5.7,173,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,Robin Hood,0.0,NaN,NaN,30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,1991,0.0,104.0,NaN,Released,NaN,5.7,26,0.00
45462,Century of Birthing,0.0,NaN,NaN,111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,NaN,2011-11-17,2011,0.0,360.0,NaN,Released,NaN,9.0,3,0.00
45463,Betrayal,0.0,NaN,NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,2003,0.0,90.0,NaN,Released,A deadly game of wits.,3.8,6,0.00
45464,Satan Triumphant,0.0,NaN,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,1917,0.0,87.0,NaN,Released,NaN,0.0,0,0.00
